## Spartans_AI-Classifican-Model-for-Diabetic-Retinopathy--DR-
- Tauha Imran
- Moaz Farooq
- Sarim Rasheed

In [3]:
# ONLY RUN IF DATA NOT DOWNLOADED!!!
# RUN THIS CELL ONLY ONCE!!!

# Importing TensorFlow - dataset
import kagglehub
# Download latest version
path = kagglehub.dataset_download("kushagratandon12/diabetic-retinopathy-balanced")

print("Path to dataset files:", path)

100%|██████████| 1.90G/1.90G [00:17<00:00, 119MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kushagratandon12/diabetic-retinopathy-balanced/versions/1


The actual model training code

In [6]:


# Importing TensorFlow
import tensorflow as tf

# Importing PyTorch
import torch

# Importing OpenCV
import cv2

# Importing FastAI
#from fastai.vision.all import *

# Importing Scikit-Learn
import sklearn

print('testing')


testing
